In [6]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image
import time
from tqdm import tqdm

# Настройка драйвера браузера
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Запуск в фоновом режиме (без UI)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Создаем папку для сохранения скриншотов, если она не существует
output_dir = 'геометрия'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Параметры для страниц
pagesize = 10  # Количество вопросов на странице
total_pages = 0  # Общее количество страниц

# Проходим по всем страницам от 0 до 332
for page_number in tqdm(range(total_pages + 1), desc="Обработка страниц", ncols=100):
    # Формируем URL для текущей страницы
    url = f'https://oge.fipi.ru/bank/questions.php?search=1&pagesize=1330&proj=DE0E276E497AB3784C3FC4CC20248DC0&theme=7%2C7.1%2C7.2%2C7.3%2C7.4%2C7.5&qlevel=&qkind=&qsstruct=&qpos=&qid=&zid=&solved=&favorite=&blind='
    
    # Открываем нужный URL
    driver.get(url)
    
    # Ждем загрузки страницы
    time.sleep(3)
    
    # Находим все элементы с классом 'qblock'
    qblocks = driver.find_elements(By.CLASS_NAME, 'qblock')
    
    # Используем tqdm для отображения прогресса по элементам на текущей странице
    for i, qblock in enumerate(qblocks):
        # Сохраняем скриншот каждого qblock элемента
        qblock_screenshot_path = os.path.join(output_dir, f'qblock_screenshot_{page_number}_{i + 1}_qblock.png')
        qblock.screenshot(qblock_screenshot_path)

        # Находим соответствующий элемент с id, который начинается с 'i' и имеет такой же код, как у qblock
        qblock_id = qblock.get_attribute('id')[1:]  # Убираем 'q' с начала
        
        try:
            # Пытаемся найти элемент с id, начинающимся с 'i'
            related_element = driver.find_element(By.ID, f'i{qblock_id}')
            
            # Сохраняем скриншот этого элемента
            related_screenshot_path = os.path.join(output_dir, f'qblock_screenshot_{page_number}_{i + 1}_related.png')
            related_element.screenshot(related_screenshot_path)

            # Открываем оба изображения с помощью Pillow
            qblock_img = Image.open(qblock_screenshot_path)
            related_img = Image.open(related_screenshot_path)

            # Объединяем два изображения по вертикали
            total_height = qblock_img.height + related_img.height
            combined_img = Image.new('RGB', (max(qblock_img.width, related_img.width), total_height))

            # Вставляем оба изображения
            combined_img.paste(qblock_img, (0, 0))
            combined_img.paste(related_img, (0, qblock_img.height))

            # Сохраняем объединенный скриншот
            combined_screenshot_path = os.path.join(output_dir, f'qblock_combined_screenshot_{page_number}_{i + 1}.png')
            combined_img.save(combined_screenshot_path)

            # Удаляем промежуточные скриншоты
            os.remove(qblock_screenshot_path)
            os.remove(related_screenshot_path)
        
        except Exception as e:
            print(f"Элемент с id 'i{qblock_id}' не найден или ошибка: {e}")
            # Просто продолжаем выполнение без сохранения второго скриншота

# Закрываем браузер
driver.quit()

print(f"Все скриншоты сохранены в папку '{output_dir}'")


Обработка страниц: 100%|████████████████████████████████████████████| 1/1 [48:08<00:00, 2888.13s/it]


Все скриншоты сохранены в папку 'геометрия'
